### Characterizing the crackle events

In [1]:
import sys

sys.path.insert(1, "/home/vinicius/storage1/projects/GrayData-Analysis")

In [2]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import xarray as xr
from frites.stats import confidence_interval
from frites.utils import parallel_func
from tqdm import tqdm
from utils import *

from config import get_dates, return_delay_split
from GDa.flatmap.flatmap import flatmap
from GDa.loader import loader
from GDa.util import shuffle_along_axis

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


##### Setting configurations

In [3]:
_ROOT = os.path.expanduser("~/funcog/gda/")

In [4]:
metric = "coh"
monkey = "lucy"
ds = 1

In [5]:
early_cue, early_delay = return_delay_split(monkey, delay_type=ds)

In [6]:
sessions = get_dates(monkey)

In [7]:
stages = [[-0.5, 0], [0, 0.5], [0.5, 1.5], [1.5, 2.0]]

### Time-resolved rate

In [8]:
import umap
from sklearn.cluster import KMeans
from sklearn.decomposition import NMF, PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score

#### Auxiliar functions

In [9]:
def bootstrap(ts_stacked, n_trials, n_rois, n_boot):
    """
    Performs the bootstrap method on the given data.

    Parameters
    ----------
    n_boot : int
        The number of iterations for the bootstrap method
    ts_stacked : ndarray
        The time series data in stacked format
    n_trials : int
        The number of trials in the time series data
    n_rois : int
        The number of regions of interest in the time series data
    verbose : bool, optional
        Whether to print the iteration number, by default False

    Returns
    -------
    ndarray
        The bootstrapped confidence interval
    """

    ci = []
    for i in range(n_boot):
        ci += [
            np.take_along_axis(
                ts_stacked,
                np.asarray(
                    [np.random.choice(range(n_trials), n_trials) for _ in range(n_rois)]
                ),
                axis=-1,
            ).mean(-1)
        ]
    ci = np.stack(ci)
    return ci

In [10]:
def compute_median_rate(
    data: xr.DataArray,
    roi: str = None,
    thr: float = 0.95,
    stim_label: int = None,
    freqs: float = None,
    time_slice: slice = None,
    # freqs: float = None,
    n_boot: int = 100,
    n_jobs: int = 1,
    verbose: bool = False,
):
    """
    Calculates the median rate of the provided xarray DataArray, subject to optional parameters.

    Parameters:
    ----------
        data: xr.DataArray
            The input data for which the median rate is to be calculated.
        roi: str | None
            A string specifying the region of interest (ROI) to consider in the analysis.
            If None, all ROIs will be considered.
        thr: (float) | .95
            A float representing the quantile based threshold.
            Any values in 'data' less than this threshold will be set to zero.
        stim_label: int | None
            An int specifying a particular stimulus label to use in the analysis.
            If None, all stimuli will be considered.
        time_slice: slice | None
            A slice object specifying a range of times to consider in the analysis.
            If None, all times will be considered.
        freqs: float | None
            A float specifying a frequency range to use in the analysis.
            If None, all frequencies will be considered.
        n_boot: int | 100
            An int specifying the number of bootstrap resamples to use when calculating
            the median rate.
        verbose: bool | False
            When set to True, the tqdm library will be used to display a progress
            bar while the function is running.

    Returns:
    -------
        Tuple of two xarray.DataArray,
        where the first is the result of median rate calculation of the given data,
        and the second is the median rate calculated by shuffling the data along the first axis.

    """

    # Check coordinates of the DataArray
    np.testing.assert_array_equal(("roi", "freqs", "trials", "times"), data.dims)

    if isinstance(stim_label, int):
        # Get stimulus label from each trial
        stim_labels = data.stim
        # Select trials with the specific label
        idx_trials = stim_labels == stim_label
    else:
        # Otherwise get all trials
        idx_trials = [True] * data.sizes["trials"]

    # Compute quantile based threshold
    thr = data.quantile(thr, ("trials", "times"))
    # Apply threshold
    data = data >= thr

    # Get time-series for specific trials, roi and time slice
    ts = data.sel(times=time_slice, roi=roi).isel(trials=idx_trials)

    if isinstance(freqs, (int, float)):
        freqs = [freqs]
    else:
        freqs = ts.freqs.data

    times = ts.times.data
    nfreqs, ntimes = len(freqs), len(times)

    def _for_freq(f):
        """
        Compute the bootstrapped confidence interval and surrogate time series data
        for a specific frequency band.

        Parameters:
        -----------
        f: float or int
            The specific frequency band for which to compute the confidence
            interval and surrogate time series data.

        Returns:
        --------
        ci: xr.DataArray
            The bootstrapped confidence interval, in the form of a xr.DataArray,
            with dimensions ('boot', 'times').
        surr: xr.DataArray
            The surrogate time series data, in the form of a xr.DataArray,
            with dimensions ('boot', 'times').
        """

        # Stack rois
        if "roi" in ts.dims:
            ts_stacked = ts.sel(freqs=f).stack(z=("trials", "roi")).data
        else:
            ts_stacked = ts.sel(freqs=f).data.T

        n_rois = ts_stacked.shape[0]
        n_trials = ts_stacked.shape[1]

        ci = bootstrap(ts_stacked, n_trials, n_rois, n_boot)

        surr = []
        for i in tqdm(range(n_boot)) if verbose else range(n_boot):
            surr += [shuffle_along_axis(ts_stacked, 0)]
        surr = np.stack(surr).mean(-1)
        ci = xr.DataArray(ci, dims=("boot", "times"), coords={"times": times})
        surr = xr.DataArray(surr, dims=("boot", "times"), coords={"times": times})

        return ci, surr

    # define the function to compute in parallel
    parallel, p_fun = parallel_func(
        _for_freq, n_jobs=n_jobs, verbose=verbose, total=nfreqs
    )
    # Compute the single trial coherence
    out = parallel(p_fun(f) for f in freqs)

    ci = [out[i][0] for i in range(nfreqs)]
    surr = [out[i][1] for i in range(nfreqs)]

    ci = xr.concat(ci, "freqs").assign_coords({"freqs": freqs})
    surr = xr.concat(surr, "freqs").assign_coords({"freqs": freqs})

    return ci, surr

#### Single-session

In [11]:
data_loader = loader(_ROOT=_ROOT)

In [12]:
kw_loader = dict(
    session="141024", aligned_at="cue", channel_numbers=False, monkey=monkey
)

power_task = data_loader.load_power(**kw_loader, trial_type=1, behavioral_response=1)
power_fix = data_loader.load_power(**kw_loader, trial_type=2, behavioral_response=0)

In [21]:
def return_burst_prob(power, conditional=False, thr=0.95):
    """
    Computes the burst probability and surrogate burst probability
    for each region of interest (ROI) in the given power dataset.

    Parameters
    ----------
    power : xarray Dataset
        A multi-dimensional dataset containing the power values of
        the time-frequency representation of the time series.
    conditional : bool, optional
        Whether to compute the burst probability and surrogate burst
        probability separately for each stimulus, by default False
    thr : float, optional
        The threshold for burst detection, by default 0.95

    Returns
    -------
    tuple
        A tuple containing the burst probability and surrogate burst
        probability for each ROI, in the format (P_b, SP_b)
    """
    kw_args = dict(
        thr=thr,
        freqs=None,
        stim_label=None,
        time_slice=slice(-0.5, 2.0),
        n_boot=100,
        verbose=False,
        n_jobs=10,
    )

    trials, stim = power.trials.data, power.attrs["stim"]
    rois = np.unique(power.roi.values)

    def _for_roi():

        # Rate modulation
        P_b = []
        SP_b = []

        for roi in tqdm(rois):
            ci, surr = compute_median_rate(power, roi=roi,  **kw_args)
            P_b += [ci]
            SP_b += [surr]

        P_b = xr.concat(P_b, "roi")
        P_b = P_b.assign_coords({"roi": rois})
        SP_b = xr.concat(SP_b, "roi")
        SP_b = SP_b.assign_coords({"roi": rois})

        return P_b, SP_b

    if not conditional:
        return _for_roi()
    else:
        # Stimulus dependent rate modulation
        P_b_stim = []
        SP_b_stim = []
        for stim in tqdm(range(1, 6)):
            kw_args["stim_label"] = stim
            P_b, SP_b = _for_roi()

            P_b_stim += [P_b]
            SP_b_stim += [SP_b]
        P_b_stim = xr.concat(P_b_stim, "stim")
        SP_b_stim = xr.concat(SP_b_stim, "stim")

        return P_b_stim, SP_b_stim

In [22]:
# Computes burst probability for task and fixation
P_b_task, SP_b_task = return_burst_prob(power_task, thr=0.95)
P_b_fix, SP_b_fix = return_burst_prob(power_fix, thr=0.95)

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          |  : 0/10 [00:00<?,       ?it/s]

  4%|▍         | 1/26 [00:04<01:45,  4.24s/it]

  0%|          |  : 0/10 [00:00<?,       ?it/s]

  8%|▊         | 2/26 [00:11<02:31,  6.30s/it]

...

 92%|█████████▏| 24/26 [00:27<00:02,  1.02s/it]

  0%|          |  : 0/10 [00:00<?,       ?it/s]

 96%|█████████▌| 25/26 [00:28<00:01,  1.08s/it]

  0%|          |  : 0/10 [00:00<?,       ?it/s]

100%|██████████| 26/26 [00:29<00:00,  1.12s/it]


In [23]:
# Computes burst probability for task and fixation
P_b_task_stim, SP_b_task_stim = return_burst_prob(power_task, conditional=True, thr=0.95)
P_b_fix_stim, SP_b_fix_stim = return_burst_prob(power_fix, conditional=True, thr=0.95)

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          |  : 0/10 [00:00<?,       ?it/s]


  4%|▍         | 1/26 [00:01<00:46,  1.85s/it]

  0%|          |  : 0/10 [00:00<?,       ?it/s]


  8%|▊         | 2/26 [00:04<00:57,  2.39s/it]

...

  0%|          |  : 0/10 [00:00<?,       ?it/s]


  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          |  : 0/10 [00:00<?,       ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]


IndexError: `indices` must be an integer array

In [228]:
X = P_b_task.median("boot").sel(freqs=27)
times = X.times.data

In [219]:
labels = KMeans(n_clusters=4, random_state=10, init="random").fit(X.data).labels_

nmf = NMF(n_components=4, init="random", random_state=0, max_iter=1000)
W = nmf.fit_transform(X.data)
H = nmf.components_

X_embedded = umap.UMAP(n_neighbors=5, min_dist=1, random_state=20).fit_transform(W)

colors = ["r", "b", "g", "m"]

times = X.times.data.astype(np.float32)

In [220]:
idx = labels.argsort()

In [221]:
def plot_rate_probability(ax, vmax=0.05):
    colors = ["r", "b", "g", "m"]

    plt.sca(ax)
    plt.imshow(
        X[idx, :], aspect="auto", cmap="turbo", origin="lower", vmin=0, vmax=vmax
    )
    cbar = plt.colorbar(extend="max")
    cbar.ax.set_ylabel(r"$P_t[B]$", rotation=90, fontsize=10)
    x_ticks_idx = ax.get_xticks()[1:-1].astype(int)
    plt.xticks(x_ticks_idx, times[x_ticks_idx])
    tks = plt.yticks(range(P_b.sizes["roi"]), P_b.roi.data[idx])
    [tks[1][i].set_color(colors[labels[idx][i]]) for i in range(P_b.sizes["roi"])]
    plt.xlabel("Time [s]", fontsize=12)
    ax.axvline(np.abs(0 - times).argmin(), -0.1, X.sizes["roi"] - 1, color="k", ls="--")
    ax.axvline(
        np.abs(0.5 - times).argmin(), -0.1, X.sizes["roi"] - 1, color="k", ls="--"
    )
    ax.axvline(
        np.abs(1.5 - times).argmin(), -0.1, X.sizes["roi"] - 1, color="k", ls="--"
    )